## Generate Heatmaps

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/covid-ma-team/chantier-2/blob/master/HeatMaps/score_heatmap-v2.ipynb"><img src="https://colab.research.google.com/img/colab_favicon_256px.png" />Run in Google Colab</a>
</td>
</table>

In [ ]:
#generate fake data
import numpy as np
import pandas as pd
import random
import sys, os
import math
import urllib.request

import folium # last version 0.11.0
from folium import plugins

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import geopandas as gpd

In [ ]:
# Load Morocco geojson (replace with the correct one)
# We will generate points insde list of regions 

morocco_map = gpd.read_file('./datasets/morocco.geojson')
irfane_map = gpd.read_file('./datasets/irfane.geojson')
regions_map = gpd.read_file('./datasets/regions_morocco.geojson')

morocco_polygon = morocco_map["geometry"][0]
irfane_polygon = irfane_map["geometry"]
regions_polygons = regions_map["geometry"]

In [ ]:
# generate a dictionaty data structure of user's latitude, longitude and score over time inside a list of regions
# input the initial position, the number of users, radius (chose a big radius to cover big regions), 
# time (integer), and list of the polygons of the regions
def generate_random_data(init_longitude = 0.0, init_latitude = 0.0, radius = 5, num_users = 1, time = 1, list_polygons = None):
    data = {}
    for ts in np.arange(time):
        new_ts = {}
        for uid in np.arange(num_users):

            u = random.random()
            v = random.random()
            w = radius * np.sqrt(u)
            t = 2 * np.pi * v
            epsilon_longitude = w * np.sin(t)
            epsilon_latitude = w * np.cos(t) 
            
            # costruct new point
            new_longitude = init_longitude + epsilon_longitude
            new_latitude = init_latitude + epsilon_latitude
            
            #check if the constructed point is inside region_polygon
            new_point = Point(new_latitude, new_longitude)

            if contains(list_polygons, new_point):
                new_longitude = init_longitude + epsilon_longitude
                new_latitude = init_latitude + epsilon_latitude
            else: # just keep initial position
                new_longitude = init_longitude
                new_latitude = init_latitude
            
            # create user data
            new_uid = {
                "uid":uid,
                "longitude": new_longitude, 
                "latitude": new_latitude, 
                "score":random.random()
                }
            new_ts[str(uid)] = new_uid
        
        # append timestamp
        data[str(ts)] = new_ts
        
    return data

In [ ]:
def contains(list_polygons, point):
    C = False
    for polygon in list_polygons:
        if polygon.contains(point):
            C = True
    return C
    

In [ ]:
#region of interest
#Test 0: Chose all morocco
LIST_POLYGONS  = [morocco_polygon]
#Test 1: Chose from the twelve regions
#  Tanger (0),Oujda (1),Rabat (2), (3), Casa (4), BeniMellal(5), Marakech(6), Errachidia(7), ...
LIST_POLYGONS  = [regions_polygons[4], regions_polygons[2]]
# Test 2: chose irfane region
LIST_POLYGONS  = irfane_polygon

# initial Points  (Rabat)
INIT_LONGITUDE = LIST_POLYGONS[1].centroid.y
INIT_LATITUDE  = LIST_POLYGONS[1].centroid.x

# simulation parameters
NUM_USERS = 1000
TIME = 100 # in seconds

In [ ]:
# Generate data
data = generate_random_data( 
    init_longitude = INIT_LONGITUDE, 
    init_latitude = INIT_LATITUDE,
    radius = .01, # radus = 0.01 for Irfane, 5 for regions, 15 for Morocco
    num_users = NUM_USERS, 
    time = TIME,
    list_polygons = LIST_POLYGONS
)

### map_points() Helper Function 
map_points takes a single time point as a dataframe of [longitude, latitude, score] and returns a map

In [ ]:
def map_points(df, lon_col='longitude', lat_col='latitude', zoom_start=20, \
                plot_points=False, pt_radius=15, \
                draw_heatmap=False, heat_map_weights_col=None, \
                heat_map_weights_normalize=True, heat_map_radius=15):
    """Creates a map given a dataframe of points. Can also produce a heatmap overlay

    Arg:
        df: dataframe containing points to maps
        lon_col: Column containing longitude (string)
        lat_col: Column containing latitude (string)
        zoom_start: Integer representing the initial zoom of the map
        plot_points: Add points to map (boolean)
        pt_radius: Size of each point
        draw_heatmap: Add heatmap to map (boolean)
        heat_map_weights_col: Column containing heatmap weights
        heat_map_weights_normalize: Normalize heatmap weights (boolean)
        heat_map_radius: Size of heatmap point

    Returns:
        folium map object
    """

    ## center map in the middle of points center in
    middle_lon = df[lon_col].median()
    middle_lat = df[lat_col].median()
    

    curr_map = folium.Map(location=[middle_lon, middle_lat],
                          zoom_start=zoom_start)

    # add points to map
    if plot_points:
        for _, row in df.iterrows():
            folium.CircleMarker([row[lon_col], row[lat_col]],
                                radius=pt_radius,
                                popup=row['uid'],
                                fill_color="#3db7e4", # divvy color
                               ).add_to(curr_map)

    # add heatmap
    if draw_heatmap:
        # convert to (n, 2) or (n, 3) matrix format
        if heat_map_weights_col is None:
            cols_to_pull = [lon_col, lat_col]
        else:
            # if we have to normalize
            if heat_map_weights_normalize:
                df[heat_map_weights_col] = \
                    df[heat_map_weights_col] / df[heat_map_weights_col].sum()

            cols_to_pull = [lon_col, lat_col, heat_map_weights_col]

        points = df[cols_to_pull].as_matrix()
        curr_map.add_child(plugins.HeatMap(points, radius=heat_map_radius))

    return curr_map

In [ ]:
# selct a time point dictionary, convert it to dataframe to be able to pass it to the map_points function
df = pd.DataFrame.from_dict(data['0'], orient='index')
df.head()

In [ ]:
map_points(df, 
           plot_points=False, 
           draw_heatmap=True, 
           heat_map_weights_normalize=True, 
           heat_map_weights_col='score',
           heat_map_radius=10)

## Heatmap Overtime

In [ ]:
# Need to reconstruct data because Folium HeatMapWithTime plugin need a list of list structure
def convert_dict_to_list_of_list(data):
    converted_data = []
    for ts in np.arange(TIME):
        list2 = []
        for key, value in data[str(ts)].items():
            list1 = []
            for k, v in value.items():
                if k!="uid":
                    list1.append(v)
            list2.append(list1)
        converted_data.append(list2)
        
    return converted_data

In [ ]:
# convert dictionary to list of list
converted_data = convert_dict_to_list_of_list(data)

# get the median position and put in the center of the map
med_lon, med_lat, med_score = np.median(np.median(np.array(converted_data), 1), 0)


In [ ]:
# create a time index
from datetime import datetime, timedelta

time_index = [
    (datetime.now() + k * timedelta(1)).strftime('%Y-%m-%d') for
    k in range(len(converted_data))
]

In [ ]:
# setup the Folium map
m = folium.Map([med_lon, med_lat],  zoom_start=20)

# plot HeatMapWithTime
hm = plugins.HeatMapWithTime(
    converted_data,
    index=time_index,
    auto_play=True,
    max_opacity=0.0
)

hm.add_to(m)

m

### Trash

In [ ]:
'''
visits = irfane_tessellation
length = len(visits['id'])
visits['num_visits'] = pd.Series(np.zeros(length), index=visits.index)

# number of sumulative visits until time t
current_time = 50
for ts in np.arange(current_time):
    #list2 = []
    for key, value in data[str(ts)].items():
        #list1 = []
        for k, v in value.items():
            if k =="latitude":
                lat = v
            if k =="longitude":
                lon = v
                
        new_point = Point(lat, lon)
        # check if the point is inside one of the irfane polygons
        for polygon in visits["geometry"]:
            print(polygon)
            if polygone.contains(new_point):
                #visits.loc[i['id'], i['num_visits']] = i['num_visits'] + 1
                print("youpi")
                break 
'''